In [2]:
def make_is_bad_zero(Xy_subset, min_interval=48, summer_start=3000, summer_end=7500):
    #Summer，3000/24=125，7500/24=312.5,The 125th day to 312.5th day would be summer

    meter = Xy_subset.meter_id.iloc[0]
    is_zero = Xy_subset.meter_reading == 0 # retunr the indices of meter reading equal to 0
    if meter == 0:
        #electric would not be 0，so drop the row（meter equals to 0）
        return is_zero

    transitions = (is_zero != is_zero.shift(1))
    all_sequence_ids = transitions.cumsum()
    ids = all_sequence_ids[is_zero].rename("ids")#return the meter equal to 0
    if meter in [2, 3]:
        # Steam and hot water could be closed
        keep = set(ids[(Xy_subset.timestamp < summer_start) |
                       (Xy_subset.timestamp > summer_end)].unique())#indices not in summer
        is_bad = ids.isin(keep) & (ids.map(ids.value_counts()) >= min_interval) 
        # Get the meter data which is closed and not in the summer(at least closed 48 hours)
    elif meter == 1:
        time_ids = ids.to_frame().join(Xy_subset.timestamp).set_index("timestamp").ids#relate ids and timestamp
        is_bad = ids.map(ids.value_counts()) >= min_interval#at least closed 48 hours

        # cold water could be closed in winter
        jan_id = time_ids.get(0, False)#id of start from Jan
        dec_id = time_ids.get(8283, False)#id of start from Des
        if (jan_id and dec_id and jan_id == time_ids.get(500, False) and
                dec_id == time_ids.get(8783, False)):
        #If the readings for both 500 hours in January and 500 hours in December are 0
            is_bad = is_bad & (~(ids.isin(set([jan_id, dec_id]))))
            #Delete this part of the row from is_bad
            
    else:
        raise Exception(f"Unexpected meter type: {meter}")

    result = is_zero.copy()
    result.update(is_bad)
    return result

def find_bad_zeros(X, y):
    """Return the Index that contains only the rows that should be deleted"""
    Xy = X.assign(meter_reading=y, meter_id=X.meter)
    is_bad_zero = Xy.groupby(["building_id", "meter"]).apply(make_is_bad_zero)
    return is_bad_zero[is_bad_zero].index.droplevel([0, 1])

In [3]:
def find_bad_sitezero(X):
    """Returns the indices of the lines with abnormal readings at Site 0."""
    return X[(X.timestamp < 3378) & (X.site_id == 0) & (X.meter == 0)].index

In [4]:
def find_bad_building1099(X, y):
    """Returns the indices of rows with abnormally high readings in building 1099 ."""
    return X[(X.building_id == 1099) & (X.meter == 2) & (y > 3e4)].index

In [5]:
def find_bad_rows(X, y):
    return find_bad_zeros(X, y).union(find_bad_sitezero(X)).union(find_bad_building1099(X, y))

In [6]:
def read_test():
    df = pd.read_csv(input_file("test.csv"), parse_dates=["timestamp"])
    df.timestamp = (df.timestamp - pd.to_datetime("2016-01-01")).dt.total_seconds() // 3600
    #This converts timestamp to the number of hours starting at 0:00 on January 1, 2016
    return compress_dataframe(df).set_index("row_id")

def read_weather_test(fix_timestamps=True, interpolate_na=True, add_na_indicators=True):
    df = pd.read_csv(input_file("weather_test.csv"), parse_dates=["timestamp"])
    df.timestamp = (df.timestamp - pd.to_datetime("2016-01-01")).dt.total_seconds() // 3600
    #This converts timestamp to the number of hours starting at 0:00 on January 1, 2016
    if fix_timestamps:
        GMT_offset_map = {site: offset for site, offset in enumerate(site_GMT_offsets)}
        df.timestamp = df.timestamp + df.site_id.map(GMT_offset_map)
        #Unify time in different time zones
    if interpolate_na:
        site_dfs = []
        for site_id in df.site_id.unique():
            # Make sure to include all possible hours, 2017 and 2018 for the entire year
            site_df = df[df.site_id == site_id].set_index("timestamp").reindex(range(8784, 26304))
            site_df.site_id = site_id
            for col in [c for c in site_df.columns if c != "site_id"]:
                if add_na_indicators: site_df[f"had_{col}"] = ~site_df[col].isna()
                site_df[col] = site_df[col].interpolate(limit_direction='both', method='linear')
                # Fill NA with median
                site_df[col] = site_df[col].fillna(df[col].median())
            site_dfs.append(site_df)
        df = pd.concat(site_dfs).reset_index()  # make timestamp back into a regular column
    elif add_na_indicators:
        for col in df.columns:
            if df[col].isna().any(): df[f"had_{col}"] = ~df[col].isna()
 #If there is a missing value in a column, a new feature is added: had_xxx indicates whether this row has a record in the xxx column
    return compress_dataframe(df).set_index(["site_id", "timestamp"])

def combined_test_data(fix_timestamps=True, interpolate_na=True, add_na_indicators=True):
    #Fill NA
    X = compress_dataframe(read_test().join(read_building_metadata(), on="building_id").join(
        read_weather_test(fix_timestamps, interpolate_na, add_na_indicators),
        on=["site_id", "timestamp"]).fillna(-1))
    return X

def combined_valid_data(fix_timestamps=True, interpolate_na=True, add_na_indicators=True):
    X = compress_dataframe(read_test().join(read_building_metadata(), on="building_id").join(
        read_weather_test(fix_timestamps, interpolate_na, add_na_indicators),
        on=["site_id", "timestamp"]).fillna(-1))
    return X

In [1]:
import pandas as pd
import numpy as np
import os
import warnings

import lightgbm as lgb
import catboost as cb
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.base import BaseEstimator, RegressorMixin, clone
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
import warnings
import copy
from tqdm import tqdm_notebook

warnings.filterwarnings('ignore')

pd.set_option("max_columns", 500)

def input_file(file):
    path = f"./{file}"
    if not os.path.exists(path): return path + ".gz"
    return path

def compress_dataframe(df):
    '''Convert all data types to numeric'''
    result = df.copy()
    for col in result.columns:
        col_data = result[col]
        dn = col_data.dtype.name
        if dn == "object":
            result[col] = pd.to_numeric(col_data.astype("category").cat.codes, downcast="integer")
        elif dn == "bool":
            result[col] = col_data.astype("int8")
        elif dn.startswith("int") or (col_data.round() == col_data).all():
            result[col] = pd.to_numeric(col_data, downcast="integer")
        else:
            result[col] = pd.to_numeric(col_data, downcast='float')
    return result

def read_train():
    df = pd.read_csv(input_file("train.csv"), parse_dates=["timestamp"])
    df.timestamp = (df.timestamp - pd.to_datetime("2016-01-01")).dt.total_seconds() // 3600
    #This converts timestamp to the number of hours starting at 0:00 on January 1, 2016
    return compress_dataframe(df)

def read_building_metadata():
    return compress_dataframe(pd.read_csv(
        input_file("building_metadata.csv")).fillna(-1)).set_index("building_id")

site_GMT_offsets = [-5, 0, -7, -5, -8, 0, -5, -5, -5, -6, -7, -5, 0, -6, -5, -5]

def read_weather_train(fix_timestamps=True, interpolate_na=True, add_na_indicators=True):
    df = pd.read_csv(input_file("weather_train.csv"), parse_dates=["timestamp"])
    df.timestamp = (df.timestamp - pd.to_datetime("2016-01-01")).dt.total_seconds() // 3600
    if fix_timestamps:
        GMT_offset_map = {site: offset for site, offset in enumerate(site_GMT_offsets)}
        df.timestamp = df.timestamp + df.site_id.map(GMT_offset_map)
        #According to the different time zones, the aligned time
    if interpolate_na:
        site_dfs = []
        for site_id in df.site_id.unique():
            # Make sure to include all possible hours
            site_df = df[df.site_id == site_id].set_index("timestamp").reindex(range(8784))
            site_df.site_id = site_id
            for col in [c for c in site_df.columns if c != "site_id"]:
                if add_na_indicators: site_df[f"had_{col}"] = ~site_df[col].isna()
                site_df[col] = site_df[col].interpolate(limit_direction='both', method='linear')
                # The missing values of some sites are filled with interpolation
                site_df[col] = site_df[col].fillna(df[col].median())
            site_dfs.append(site_df)
        df = pd.concat(site_dfs).reset_index()  
        # Before setting timestamp to index, reset index here to make timestamp return to the general column
    elif add_na_indicators:
        for col in df.columns:
            if df[col].isna().any(): df[f"had_{col}"] = ~df[col].isna()
    return compress_dataframe(df).set_index(["site_id", "timestamp"])

def combined_train_data(fix_timestamps=True, interpolate_na=True, add_na_indicators=True):
    Xy = compress_dataframe(read_train().join(read_building_metadata(), on="building_id").join(
        read_weather_train(fix_timestamps, interpolate_na, add_na_indicators),
        on=["site_id", "timestamp"]).fillna(-1))
    return Xy.drop(columns=["meter_reading"]), Xy.meter_reading

def _add_time_features(X):
    return X.assign(tm_day_of_week=((X.timestamp // 24) % 7), tm_hour_of_day=(X.timestamp % 24))

categorical_columns = [
    "building_id", "meter", "site_id", "primary_use", "had_air_temperature", "had_cloud_coverage",
    "had_dew_temperature", "had_precip_depth_1_hr", "had_sea_level_pressure", "had_wind_direction",
    "had_wind_speed", "tm_day_of_week", "tm_hour_of_day"
]


In [9]:
X, y = combined_train_data()

bad_rows = find_bad_rows(X, y)
pd.Series(bad_rows.sort_values()).to_csv("rows_to_drop.csv", header=False, index=False)

Xy = X.assign(meter_reading=y, meter_id=X.meter)

In [9]:
X = X.drop(index=bad_rows)
y = y.reindex_like(X)

# Additional preprocessing
X = compress_dataframe(_add_time_features(X))
X = X.drop(columns="timestamp")  # drop the original timestamp
y = np.log1p(y)

In [ ]:
X.columns.values

In [ ]:
params = {
    'task': 'train',
    'boosting_type': 'gbdt',  
    'objective': 'regression',  
    'metric': 'rmse',  
    'num_leaves': 1280,  
    'learning_rate': 0.01,  
    #'feature_fraction': 0.8,  
    #'bagging_fraction': 0.8,  
    #'bagging_freq': 5,  
    'verbose': 1,
    #'max_depth': 8,
    #'lambda_l1':1,
    #'lambda_l2':3,
}

fitted={}
for val in X['meter'].unique():
    X1 = X[X['meter'] == val].drop(columns=['meter'])
    kf = StratifiedKFold(n_splits=3,random_state=42)
    #Use StratifiedKFold to make the distribution of the specified column in each fold the same, here is divided into 3 folds
    fitted[val]=[]
    t=0
    for train_index,test_index in kf.split(X1,X1['tm_hour_of_day']):
        #Make the distribution of ['tm_hour_of_day'] in each fold the same
        train_features = X1.iloc[train_index]
        train_target = y[X1.iloc[train_index].index]
        
        test_features = X1.iloc[test_index]
        test_target = y[X1.iloc[test_index].index]
        
        d_train = lgb.Dataset(train_features, train_target, categorical_feature=categorical_features)
        d_eval = lgb.Dataset(test_features,test_target, categorical_feature=categorical_features)
        print("Building model meter :",val,'fold:',t)
        t+=1
        
        md = lgb.train(params, d_train, num_boost_round=20000, valid_sets=d_eval, early_stopping_rounds=500,verbose_eval=200)
        #It can also be predicted directly below, without retaining the model, saving menmory
        model=copy.deepcopy(md)
        #Use deep copy to change the location where the model is saved
        fitted[val].append(model)
del X, y,X1,train_features,test_features

In [11]:
X = combined_test_data()
X = compress_dataframe(_add_time_features(X))
X = X.drop(columns="timestamp")  # drop the original timestamp

In [ ]:
result = np.zeros(len(X))
for val in X['meter'].unique():
    ix = np.nonzero((X['meter'] == val).to_numpy())
    predictions=np.zeros([1,len(ix[0])])
    for i in tqdm_notebook(range(len(fitted[val]))):
        #fitted is a dictionary, fitted [val] is a list, and stores the trained model, where val stands for meter type
        model=fitted[val][i]
        predictions += model.predict(X.iloc[ix].drop(columns=['meter']),num_iteration=model.best_iteration)
    result[ix] = predictions/len(fitted[val])

In [13]:
predictions = pd.DataFrame({
    "row_id": X.index,
    "meter_reading": np.clip(np.expm1(result), 0, None)
})

In [14]:
predictions.to_csv("cleanup_Klgb_12.csv", index=False, float_format="%.4f")